<a href="https://colab.research.google.com/github/cristiandarioortegayubro/BDS/blob/main/algoritmos/bds_gradient_boosting_001_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20Scikit-learn.png?raw=true">
</p>

 # **<font color="DeepPink">Gradient Boosting para clasificación</font>**


<p align="justify">
En el colab anterior, exploramos el uso de algoritmos de Gradient Boosting aplicados a problemas de regresión. En este nuevo colab, nos enfocaremos en el algoritmo <code>GradientBoostingClassifier</code> y su aplicación en problemas de clasificación.
<br><br>
Recuerde que el Gradient Boosting es una técnica de aprendizaje automático que combina múltiples modelos más simples (generalmente árboles de decisión) para construir un modelo predictivo más poderoso. En el caso específico del <code>GradientBoostingClassifier</code>, nos permitirá abordar tareas de clasificación, donde la variable respuesta es de naturaleza categórica.  

## **<font color="DeepPink">Bibliotecas**

In [ ]:
# Operaciones matemáticas y estadísticas
import pandas as pd
import numpy as np

In [ ]:
# Visualización
import plotly.express as px
import plotly.graph_objs as go

## **<font color="DeepPink">Conjunto de Datos**

<p align="justify">
El conjunto de datos <code>Carseats</code>, original del paquete de R <code>ISLR</code> y accesible en Python a través de <code>statsmodels.datasets.get_rdataset</code>, contiene información sobre la venta de sillas infantiles en 400 tiendas distintas.
<br>
<br>
Para cada una de las 400 tiendas se han registrado 11 variables. Se pretende generar un modelo de clasificación que permita predecir si una tienda tiene ventas altas (<code>Sales</code> $>$ 8) o bajas (<code>Sales</code> $<=$ 8) en función de todas las variables disponibles.

In [ ]:
import statsmodels.api as sm
carseats = sm.datasets.get_rdataset("Carseats", "ISLR")
datos = carseats.data
#print(carseats.__doc__)

In [ ]:
datos.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [ ]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 34.5+ KB


<p align="justify">
Como <code>Sales</code> es una variable continua y el objetivo del estudio es clasificar las tiendas según si venden mucho o poco, se crea una nueva variable dicotómica (<code>'altas'</code>, <code>'bajas'</code>) llamada <code>sales</code>.

In [ ]:
datos['Sales'] = datos.Sales.apply(lambda x: "altas" if x > 8 else "bajas")
#datos['Sales'] = np.where(datos.Sales > 8, "altas", "bajas")

In [ ]:
datos

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,altas,138,73,11,276,120,Bad,42,17,Yes,Yes
1,altas,111,48,16,260,83,Good,65,10,Yes,Yes
2,altas,113,35,10,269,80,Medium,59,12,Yes,Yes
3,bajas,117,100,4,466,97,Medium,55,14,Yes,Yes
4,bajas,141,64,3,340,128,Bad,38,13,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...
395,altas,138,108,17,203,128,Good,33,14,Yes,Yes
396,bajas,139,23,3,37,120,Medium,55,11,No,Yes
397,bajas,162,26,12,368,159,Medium,40,18,Yes,Yes
398,bajas,100,79,7,284,95,Bad,50,12,Yes,Yes


 ## **<font color="DeepPink">División del conjunto de datos</font>**

In [ ]:
X = datos.drop(columns=['Sales'])
y = datos['Sales']

 ## **<font color="DeepPink">Preprocesamiento de variables categóricas con `sklearn`</font>**

<p align="justify">
A diferencia del colab anterior, en estos datos hay variables explicativas categóricas por lo que, antes de entrenar el modelo, es necesario aplicar <i>one-hot-encoding</i>.

In [ ]:
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

<p align="justify">
Primero se identifica el nombre de las columnas categóricas y numéricas. El resultado es una <code>lista</code>.

In [ ]:
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(X)

In [ ]:
numerical_columns_selector = selector(dtype_include=[int,float])
numerical_columns = numerical_columns_selector(X)

<p align="justify">
Luego se aplica <i>one-hot-encoding</i> solo a las columnas categóricas. El parámetro <code>remainder</code> en <code>ColumnTransformer</code> determina cómo se deben manejar las columnas que no son seleccionadas o transformadas por los transformadores.
<br>
<br>
De forma predeterminada, el parámetro <code>remainder</code> está configurado en <code>drop</code>, lo que significa que cualquier columna restante que no esté especificada en los transformadores se eliminará.
<br>
<br>
Alternativamente, puedes establecer <code>remainder='passthrough'</code> para incluir las columnas restantes en la salida sin aplicar ninguna transformación. Esto es útil cuando se necesitan mantener ciertas columnas sin cambios, como en este caso las columnas numéricas.

In [ ]:
preprocessor = ColumnTransformer(
                    [('one-hot-encoding',
                      OneHotEncoder(handle_unknown='ignore',
                                    sparse_output=False),
                      categorical_columns)],
                    remainder='passthrough')

<p align="justify">
Una vez que se ha definido el objeto <code>ColumnTransformer</code>, con el método <code>fit_transform()</code> se aplican las tranformaciones al conjunto de datos <code>X</code>.

In [ ]:
X_encoded = preprocessor.fit_transform(X)
X_encoded

array([[  1.,   0.,   0., ..., 120.,  42.,  17.],
       [  0.,   1.,   0., ...,  83.,  65.,  10.],
       [  0.,   0.,   1., ...,  80.,  59.,  12.],
       ...,
       [  0.,   0.,   1., ..., 159.,  40.,  18.],
       [  1.,   0.,   0., ...,  95.,  50.,  12.],
       [  0.,   1.,   0., ..., 120.,  49.,  16.]])

<p align="justify">
El resultado devuelto es un <code>numpy array</code>, por lo que se pierden los nombres de las columnas. Suele ser interesante poder inspeccionar cómo queda el conjunto de datos tras el preprocesado en formato <code>DataFrame</code>.
<br>
<br>
Por defecto, <code>OneHotEncoder</code> ordena las nuevas columnas de izquierda a derecha por orden alfabético.

Convertir el `numpy array` en `dataframe` y añadir el nombre de las columnas.

In [ ]:
columns_endoded = preprocessor.named_transformers_['one-hot-encoding'].get_feature_names_out(categorical_columns)
columns_endoded

array(['ShelveLoc_Bad', 'ShelveLoc_Good', 'ShelveLoc_Medium', 'Urban_No',
       'Urban_Yes', 'US_No', 'US_Yes'], dtype=object)

In [ ]:
labels = np.concatenate([columns_endoded,numerical_columns])
X_transformed = pd.DataFrame(X_encoded, columns=labels)
X_transformed.head()

,ShelveLoc_Bad,ShelveLoc_Good,ShelveLoc_Medium,Urban_No,Urban_Yes,US_No,US_Yes,CompPrice,Income,Advertising,Population,Price,Age,Education
0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,138.0,73.0,11.0,276.0,120.0,42.0,17.0
1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,111.0,48.0,16.0,260.0,83.0,65.0,10.0
2,0.0,0.0,1.0,0.0,1.0,0.0,1.0,113.0,35.0,10.0,269.0,80.0,59.0,12.0
3,0.0,0.0,1.0,0.0,1.0,0.0,1.0,117.0,100.0,4.0,466.0,97.0,55.0,14.0
4,1.0,0.0,0.0,0.0,1.0,1.0,0.0,141.0,64.0,3.0,340.0,128.0,38.0,13.0


In [ ]:
X_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ShelveLoc_Bad     400 non-null    float64
 1   ShelveLoc_Good    400 non-null    float64
 2   ShelveLoc_Medium  400 non-null    float64
 3   Urban_No          400 non-null    float64
 4   Urban_Yes         400 non-null    float64
 5   US_No             400 non-null    float64
 6   US_Yes            400 non-null    float64
 7   CompPrice         400 non-null    float64
 8   Income            400 non-null    float64
 9   Advertising       400 non-null    float64
 10  Population        400 non-null    float64
 11  Price             400 non-null    float64
 12  Age               400 non-null    float64
 13  Education         400 non-null    float64
dtypes: float64(14)
memory usage: 43.9 KB


 ## **<font color="DeepPink">División del conjunto de entrenamiento y prueba</font>**

👀 Dividimos en conjunto de entrenamiento y prueba

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_transformed,
                                                    y,
                                                    random_state=123)

 # **<font color="DeepPink">Gradient Boosting para clasificación</font>**

 ## **<font color="DeepPink">Cración y ajuste del modelo</font>**

Se ajusta un modelo empleando como variable respuesta `Sales` y como variables predictoras todas las otras variables disponibles.

La clase `GradientBoostingClassifier` del módulo `ensemble` permite entrenar modelos Gradient Boosting para problemas de clasificación. Puede encontrarse una descripción detallada de todos los hiperparámetros en [sklearn.ensemble.GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html). Algunos de sus hiperparámetros más importantes son:

- `learning_rate`: es el porcentaje de cambio con el que se actualizan los coeficientes de pesos en cada iteración. Es decir, cada vez que se realiza una iteración en el proceso de entrenamiento se actualizan los pesos de las variables explicativas para dar cada vez una mejor aproximación.

- `n_estimators`: es el número de árboles incluidos en el modelo.

- `max_depth`: es la profundidad máxima que pueden alcanzar los árboles.

- `min_samples_split`: es el número mínimo de muestras o datos que debe de tener un nodo para que pueda dividirse.

- `min_samples_leaf`: es el número mínimo de muestras requeridas en un nodo hoja.

- `max_leaf_nodes`: es el número máximo de nodos hoja para cada árbol.

- `max_features`: es el número de variables predictoras consideradas al buscar la mejor división para construir cada árbol de decisión. Puede ser:

    - Un valor entero.
    - Una fracción del total de predictores.
    - “sqrt”, raiz cuadrada del número total de predictores.
    - “log2”, log2 del número total de predictores.
    - None, utiliza todos los predictores.

Los que determinan la parada temprana (early stopping):

- `tol`: es el porcentaje mínimo de mejora entre dos iteraciones consecutivas por debajo del cual se considera que el modelo no ha mejorado.

- `n_iter_no_change`: representa el número de iteraciones consecutivas en las que no se produce una mejora significativa en el rendimiento del modelo para que el algoritmo se detenga (parada temprana). Si su valor es None se desactiva la parada temprana.

- `validation_fraction`: proporción de datos separados del conjunto entrenamiento y empleados como conjunto de validación para determinar la parada temprana.

<p align="justify">
La parada temprana es una estrategia para evitar un entrenamiento prolongado innecesario y ahorrar tiempo de cómputo cuando el modelo ya ha alcanzado su mejor rendimiento y no está mejorando significativamente. Por otro lado, evita el sobreajuste y mejora la generalización del modelo a nuevos datos.


<p align="justify">
Definimos una grilla de hiperparámetros para ajustar un modelo <code>GradientBoostingClassifier</code>.
<br><br>
Los hiperparámetros considerados son:

- <code>n_estimators</code> : representa el número de árboles que se construirán. Los valores son: 50, 100 y 500.

- <code>max_features</code> : número máximo de variables explicativas que se considerarán al buscar la mejor división para construir cada árbol. "None" indica que todas las características se considerarán, mientras que "sqrt" y "log2" representan la raíz cuadrada y el logaritmo en base 2 del número total de variables predictoras, respectivamente.

- <code>max_depth</code> : representa la profundidad máxima de los árboles. Los valores posibles son None, 1, 3, 5, 10 y 20.

- <code>learning_rate</code> : el algoritmo probará tres tasas de aprendizaje diferentes: 0.001, 0.01 y 0.1.

Durante la búsqueda de hiperparámetros, se probarán todas las combinaciones posibles de estos valores para encontrar la combinación óptima que resulte en el mejor rendimiento del modelo.

In [ ]:
# Grilla de hiperparámetros evaluados
param_grid = {'n_estimators'  : [50, 100, 500],
              'max_features'  : [None, 'sqrt', 'log2'],
              'max_depth'     : [None, 1, 3, 5, 10, 20],
              'learning_rate' : [0.001, 0.01, 0.1]
             }

<p align="justify">
La clase <code>GridSearchCV</code> de <code>sklearn</code> ajusta los hiperparámetros del modelo para encontrar la combinación óptima que produce el mejor rendimiento en nuevos datos.
<br><br>
Primero se crea una instancia de la clase <code>GridSearchCV</code>. Sus parámetros son:

- <code>estimator</code> : es el modelo que se utilizará en la búsqueda de hiperparámetros. En este caso, se utiliza un <code>GradientBoostingClassifier</code> con algunos hiperparámetros que activan la parada temprana.
- <code>param_grid</code> : es el diccionario de hiperparámetros que se desea ajustar. En este caso, el diccionario <code>param_grid</code> contiene todas las combinaciones de valores posibles que se probarán para los hiperparámetros del modelo <code>GradientBoostingClassifier</code>.
- <code>scoring</code> : es la métrica que se utilizará para evaluar y comparar los modelos con diferentes hiperparámetros. En este caso, se utiliza la métrica "accuracy".
- <code>n_jobs</code> : la cantidad de núcleos de CPU que se utilizarán para realizar la búsqueda de hiperparámetros en paralelo. Se establece en <code>multiprocessing.cpu_count() - 1</code>, lo cual significa que utilizarán todos los núcleos disponibles menos uno.
- <code>refit</code> : si se establece en <code>True</code>, después de encontrar la mejor combinación de hiperparámetros, se reentrenará el modelo utilizando el conjunto completo de entrenamiento.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
import multiprocessing

grid = GridSearchCV(estimator  = GradientBoostingClassifier(random_state=123,
                                                            # Activación de la parada temprana
                                                            tol                 = 0.0001,
                                                            validation_fraction = 0.1,
                                                            n_iter_no_change    = 5),
                    param_grid = param_grid,
                    scoring    = 'accuracy',
                    n_jobs     = multiprocessing.cpu_count() - 1,
                    refit      = True)

grid.fit(X_train, y_train)

GridSearchCV(estimator=GradientBoostingClassifier(n_iter_no_change=5,
                                                  random_state=123),
             n_jobs=1,
             param_grid={'learning_rate': [0.001, 0.01, 0.1],
                         'max_depth': [None, 1, 3, 5, 10, 20],
                         'max_features': [None, 'sqrt', 'log2'],
                         'n_estimators': [50, 100, 500]},
             scoring='accuracy')

<p align="justify">
Podemos inspeccionar todos los resultados que se almacenan en el atributo <code>cv_results_</code> de la búsqueda en grid-search. Eliminamos algunas columnas de estos resultados.

In [ ]:
cv_results = pd.DataFrame(grid.cv_results_).sort_values('mean_test_score', ascending = False)
cv_results = cv_results.drop(columns=['std_test_score','rank_test_score','params','split0_test_score','split1_test_score','split2_test_score','split3_test_score','split4_test_score','mean_fit_time','std_fit_time','mean_score_time','std_score_time'])
cv_results.head()

,param_learning_rate,param_max_depth,param_max_features,param_n_estimators,mean_test_score
118,0.1,1,None,100,0.836667
119,0.1,1,None,500,0.836667
126,0.1,3,None,50,0.823333
128,0.1,3,None,500,0.820000
127,0.1,3,None,100,0.820000


<p align="justify">
Es posible obtener los hiperparámetros óptimos utilizando el atributo <code>best_params_</code> de la búsqueda en grid-search. Además, mediante los atributos <code>best_score_</code> y <code>scoring</code>, podemos acceder al valor del rendimiento óptimo y a la métrica de evaluación respectivamente, asociados con los hiperparámetros obtenidos.

In [ ]:
print("Mejores hiperparámetros encontrados")
print(grid.best_params_, ":", grid.best_score_, grid.scoring)

Mejores hiperparámetros encontrados
{'learning_rate': 0.1, 'max_depth': 1, 'max_features': None, 'n_estimators': 100} : 0.8366666666666667 accuracy


<p align="justify">
Una vez identificados los mejores hiperparámetros, se reentrena el modelo indicando los valores óptimos en sus argumentos. Si en el <code>GridSearchCV</code> se indica <code>refit=True</code>, este reentrenamiento se hace automáticamente y el modelo resultante se encuentra almacenado en el atributo <code>best_estimator_</code>.

In [ ]:
final_model = grid.best_estimator_

 ## **<font color="DeepPink">Predicción y evaluación del modelo</font>**

<p align="justify">
Por último, se evalúa la capacidad predictiva del modelo final empleando el conjunto de prueba.

In [ ]:
# Predicción de probabilidades
prob_prediction = final_model.predict_proba(X = X_test)
prob_prediction[:5, :]

array([[0.58466318, 0.41533682],
       [0.5535139 , 0.4464861 ],
       [0.52896978, 0.47103022],
       [0.49292479, 0.50707521],
       [0.47880108, 0.52119892]])

<p align="justify">
El resultado de <code>predict_proba</code> es un array con una fila por observación y tantas columnas como clases tenga la variable respuesta. El valor de la primera columna se corresponde con la probabilidad, acorde al modelo, de que la observación pertenezca a la clase <code>altas</code> y la segunda columna a la clase <code>bajas</code>.

In [ ]:
prediction = final_model.predict(X = X_test)
prediction[:10]

array(['altas', 'altas', 'altas', 'bajas', 'bajas', 'bajas', 'bajas',
       'bajas', 'bajas', 'bajas'], dtype=object)

<p align="justify">
Por defecto, <code>predict</code> asigna cada nueva observación a la clase con mayor probabilidad (en caso de empate se asigna de forma aleatoria). Sin embargo, este no tiene por qué ser el comportamiento deseado en todos los casos.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true = y_test,
                      y_pred = prediction)

accuracy = accuracy_score(y_true = y_test,
                          y_pred = prediction,
                          normalize = True)

<p align="justify">
El método <code>accuracy_score</code> es una métrica de evaluación utilizada para calcular la precisión de un modelo de clasificación. Esta métrica determina la proporción de muestras clasificadas correctamente en relación con el total de muestras evaluadas. Por otro lado, <code>confusion_matrix</code> se encarga de generar la matriz de confusión, la cual ofrece información relevante acerca de cómo el modelo clasifica de forma correcta o incorrecta las muestras en distintas clases.
<br><br>
Al aplicar estas métricas de evaluación, los profesionales pueden determinar la eficacia del modelo en la tarea de clasificación y, a su vez, identificar posibles áreas de mejora. Esto permite tomar decisiones informadas y optimizar el desempeño del modelo a fin de lograr resultados más precisos y confiables en aplicaciones prácticas.



In [ ]:
print("Matriz de confusión")
print("-------------------")
print(cm)
print("")
print(f"La accuracy de modelo final es: {100 * accuracy} %")

Matriz de confusión
-------------------
[[28 22]
 [ 1 49]]

La accuracy de modelo final es: 77.0 %


<p align="justify">
Tras optimizar los hiperparámetros, la <b>exactitud</b> o <b>accuracy</b> del modelo es 0.77. Es decir, el modelo es capaz de predecir correctamente un 77 % de las observaciones del conjunto de prueba.

 ## **<font color="DeepPink">Importancia de las variables predictoras</font>**

<p align="justify">
El atributo <code>feature_importances_</code> es específico de algunos modelos de aprendizaje automático, como el Gradient Boosting, y se utiliza para obtener la importancia de las variables explicativas utilizadas por el modelo para realizar predicciones.

In [ ]:
importancia_predictores = pd.DataFrame(
                            {'predictor': X_transformed.columns,
                             'importancia': final_model.feature_importances_}
                            )
importancia_predictores

,predictor,importancia
0,ShelveLoc_Bad,0.147649
1,ShelveLoc_Good,0.301519
2,ShelveLoc_Medium,0.000000
3,Urban_No,0.000000
4,Urban_Yes,0.000000
5,US_No,0.000000
6,US_Yes,0.000000
7,CompPrice,0.010232
8,Income,0.042184
9,Advertising,0.173977


In [ ]:
fig = px.bar(importancia_predictores.sort_values(by='importancia'),
             x = 'importancia',
             y = 'predictor')

fig.update_layout(template="gridon",
                  title_text="Importancia de las variables predictoras",
                  xaxis_title='',
                  yaxis_title='',
                  yaxis=dict(tickfont=dict(size=8))
                  )

fig.show()

<p align="justify">
La importancia de una variable explicativa se mide por la magnitud de su contribución al rendimiento general del modelo. En este caso, <code>ShelveLoc_God</code> (buena ubicación en góndola), <code>Price</code> y <code>Advertising</code> son responsables de aproximadamente el 75 % de la capacidad predictiva del modelo.

 # **<font color="DeepPink">Conclusiones</font>**

<p align="justify">
👀 En este colab nosotros:
<br><br>
✅ Realizamos la codificación de variables categóricas mediante <code>OneHotEncoder</code>.
<br>
✅ Utilizamos la biblioteca <code>scikit_learn</code> para entrenar un modelo de gradient boosting en el contexto de un problema de clasificación.
<br>
✅ Optimizamos los hiperparámetros de un modelo predictivo a través del grid-search.
<br>
✅ Analizamos la importancia de las variables explicativas mediente el atributo <code>feature_importances_</code>.


<br>
<br>
<p align="center"><b>
💗
<font color="DeepPink">
Hemos llegado al final de nuestro colab, a seguir codeando...
</font>
</p>
<br>
<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>

---
